# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#Answer for question 1

---

Yes, averaging the validation accuracy across multiple splits can indeed provide more consistent and reliable results when assessing the performance of a machine learning model. This technique is commonly known as cross-validation, and it helps to address the potential variability in model performance that can arise from a single train-validation split. Here's how it works:

K-Fold Cross-Validation: The most common form of cross-validation is k-fold cross-validation. In this method, the dataset is divided into k subsets or "folds" of approximately equal size. The model is then trained and validated k times, with each fold serving as the validation set once while the remaining k-1 folds are used for training. This process yields k different validation accuracy scores.

Averaging Results: After each of the k iterations, you have k different validation accuracy scores. These scores can be averaged to obtain a single, more robust estimate of the model's performance. This average is often considered a more stable and reliable measure of performance compared to a single validation set.

Reduced Variability: Cross-validation helps reduce the impact of randomness and variability that can occur when splitting data into a single train-validation set. It provides a better representation of the model's generalization performance across different subsets of the data.

Hyperparameter Tuning: Cross-validation is particularly useful during hyperparameter tuning (e.g., tuning the learning rate, regularization strength, or model architecture). It allows you to assess how different hyperparameters impact model performance across multiple data splits, helping you make more informed choices.

Model Selection: Cross-validation can also be used for model selection. You can compare the cross-validated performance of different models to determine which one performs best on average across various data partitions.

Stratified Cross-Validation: To ensure that each class in a classification problem is represented in the validation sets proportionally, you can use stratified k-fold cross-validation.

Overall, cross-validation is a valuable technique for obtaining more stable and consistent estimates of a model's performance. It is especially useful when working with limited data or when you want to make more confident assertions about how well your model is likely to perform on unseen data. Common values for k include 5-fold and 10-fold cross-validation, but the choice of k can depend on the size of your dataset and computational resources.

#Answer for question 2
---

Cross-validation, such as k-fold cross-validation, provides a more accurate estimate of test accuracy compared to a single train-validation split when you have a limited dataset. However, it's important to understand the distinction between "test accuracy" and "cross-validated accuracy."

Test Accuracy: The "test accuracy" typically refers to the performance of your model on a completely independent and unseen dataset that is separate from your training and validation data. It is the ultimate measure of how well your model generalizes to new, real-world data.

Cross-Validated Accuracy: Cross-validated accuracy is an estimate of your model's performance based on multiple rounds of training and validation on different subsets of your training data. It provides a more stable and reliable estimate of how your model is likely to perform on unseen data compared to a single validation split.

Here's how cross-validation relates to test accuracy:

Cross-validated accuracy is a valuable metric for assessing your model's performance and comparing different models during development and hyperparameter tuning. It helps you make informed decisions about model selection and tuning.

While cross-validated accuracy provides a better estimate of a model's generalization performance compared to a single validation split, it is still an estimate based on your training data. It does not replace the need for a true test set to evaluate how well your model performs in the real world.

The true test accuracy, measured on a separate and previously unseen dataset, is what ultimately tells you how your model is expected to perform in production or on new data. Cross-validated accuracy is a useful intermediate step to assess your model's capabilities and guide your development process, but it may not precisely match the test accuracy.

In summary, cross-validation does not replace the need for a dedicated test set but provides a more accurate and reliable estimate of your model's performance compared to a single validation split. It is a crucial tool for model development and evaluation, especially when you have limited data. To get the true test accuracy, you should reserve a separate test dataset that is not used during model development or hyperparameter tuning.

#Answer for question 3
---

The number of iterations or folds in cross-validation can have an effect on the accuracy estimate, and the impact depends on various factors. Generally, increasing the number of iterations (i.e., using more folds) can provide a more stable and robust estimate of a model's performance, but there are some trade-offs to consider:

Pros of Using More Iterations:

Stability: With more iterations, the estimate of model performance becomes more stable and less sensitive to the specific random split of the data. This can help reduce the variability in your performance estimate.

Better Utilization of Data: More iterations allow you to make better use of your available data for both training and validation. This can be especially important when you have a limited dataset.

Improved Hyperparameter Tuning: When conducting hyperparameter tuning, more iterations provide a more comprehensive assessment of how different hyperparameters affect model performance, leading to better parameter choices.

Cons and Considerations:

Computational Cost: Using a higher number of iterations increases the computational cost of cross-validation, as you need to train and evaluate the model multiple times. This can be a significant factor when working with large datasets or complex models.

Diminishing Returns: While increasing the number of iterations can improve the estimate's stability, there are diminishing returns. After a certain point, adding more folds may not substantially improve the estimate but will significantly increase the computation time.

Imbalanced Data: In cases where your data is highly imbalanced, using a very high number of folds can lead to issues where some folds contain very few or no instances of certain classes. This can result in biased performance estimates.

Small Datasets: If your dataset is very small, you might need to be cautious about using a large number of folds because it can lead to each fold having too few samples for training, potentially resulting in poor model training.

In practice, common choices for the number of iterations in k-fold cross-validation include 5-fold and 10-fold cross-validation. These values strike a balance between stability and computational cost for many datasets. However, the optimal number of iterations can vary depending on your specific dataset and goals.
In summary, increasing the number of iterations in cross-validation can lead to a more stable and robust estimate of model performance, but there are trade-offs to consider, including computational cost and potential issues with imbalanced or small datasets. The choice of the number of iterations should be based on practical considerations and the characteristics of your data.

#Answer for question 4
---


Increasing the number of iterations in cross-validation can help mitigate some of the challenges associated with having a very small training dataset or validation dataset to some extent, but it may not completely resolve all issues. Here's how increasing iterations can help in such situations:

Benefits of Increasing Iterations with Small Datasets:

Better Utilization of Data: With more iterations, you can partition your small dataset into different folds more times. This can help in better utilizing the limited data for training and validation.

Stability: More iterations provide a more stable estimate of model performance, reducing the impact of randomness in the data split. This can make the performance estimate less sensitive to the specific choice of the validation set.

Enhanced Hyperparameter Tuning: If you're conducting hyperparameter tuning, having more iterations can help you explore a wider range of hyperparameters and make more informed decisions about the best configuration.

However, there are limitations and considerations:

Imbalanced Data: If your dataset is not only small but also imbalanced (i.e., certain classes have very few samples), increasing the iterations might lead to some folds with extremely imbalanced class distributions. This can affect the reliability of the performance estimate, especially for minority classes.

Computational Cost: Using a high number of iterations can significantly increase computational cost, as each iteration involves training and evaluating the model. This might be a concern if you have limited computational resources or time constraints.

Reduced Training Data per Fold: When you have a very small dataset, increasing the number of iterations means that each fold will have an even smaller portion of the data for training. This can lead to issues with model training and may result in poor model generalization.

In cases where you have a very small training dataset, it's important to carefully consider the trade-offs. While increasing the iterations can help, it may not fully compensate for the limitations imposed by a small dataset. It's also worth exploring other techniques to address data scarcity, such as data augmentation, transfer learning, or, if possible, obtaining more data.